In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from toolz.curried import *
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from util import *

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch
from torch.utils.data import DataLoader

from pytorch_forecasting import Baseline, NBeats, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE
from dataloader import subset_sampler

In [5]:
from models import RNN

In [6]:
device = torch.device("cuda:0")
best_model_path="/home_nfs/jiangyue/github/carpark-prediction/lightning_logs/version_33/checkpoints/epoch=2-step=1499.ckpt"
best_model = RNN.load_from_checkpoint(best_model_path).to(device)
print(best_model_path)
print(best_model.device)

/home_nfs/jiangyue/github/carpark-prediction/lightning_logs/version_33/checkpoints/epoch=2-step=1499.ckpt
cuda:0


In [7]:
year, month = 2021, 6
carpark_dir = "/home_nfs/jiangyue/data/carparking"
month_dir = os.path.join(carpark_dir, f"{year}/{month:02}")
carpark_meta = load_carpark_meta(os.path.join(carpark_dir, "carpark_meta_present.geojson"))

In [8]:
carpark_data_names = [
    "carpark_data_2021_06_17-18.json",
    #"carpark_data_2021_04_21-25.json",
    #"carpark_data_2021_04_26-30.json"
]

carpark_data = load_carpark_data_list(
    [os.path.join(carpark_dir, name) for name in carpark_data_names]
)

Loading /home_nfs/jiangyue/data/carparking/carpark_data_2021_06_17-18.json


In [9]:
carpark_data = carpark_remove_outlier(carpark_data)
carpark_data = carpark_data.fillna(method="ffill")

100%|██████████| 1918/1918 [01:26<00:00, 22.22it/s]


In [10]:
carpark_data = carpark_add_date(carpark_data)
carpark_data["min"] = (carpark_data.tid % 1440 // 5).astype(str)
len(carpark_data.sid.unique())

1918

In [11]:
tid_start=len(carpark_data.tid.unique())
tid_end=tid_start+119
sid_start=0
sid_end=len(carpark_data.sid.unique())-1
date_info=carpark_data.iloc[tid_start-1].at['date']
carpark_data_c=carpark_data.copy()
date_data=pd.date_range(date_info, periods=121, freq='T')[1:]
sid_list=range(len(carpark_data.sid.unique()))

In [12]:
for x in sid_list:
    data={"sid":[x]*120,"tid":range(tid_start, tid_end+1),"AvailableLots":[0]*120, 'date':date_data }
    df=pd.DataFrame(data,index=range(120))
    df["min"] = (df.tid % 1440 // 5).astype(str)
    frames = [carpark_data_c, df]
    carpark_data_c = pd.concat(frames)

In [13]:
carpark_data_c=carpark_data_c.reset_index(drop=True)

In [14]:
carpark_data_c['Weekday']=carpark_data_c['date'].dt.dayofweek.astype(str)

In [15]:
carpark_data_c.head()

,sid,tid,AvailableLots,date,min,Weekday
0,0,0,2786.0,2021-06-17 00:00:00,0,3
1,0,1,2787.0,2021-06-17 00:01:00,0,3
2,0,2,2787.0,2021-06-17 00:02:00,0,3
3,0,3,2787.0,2021-06-17 00:03:00,0,3
4,0,4,2787.0,2021-06-17 00:04:00,0,3


In [16]:
print(carpark_data_c[carpark_data_c.tid==0])

          sid  tid  AvailableLots       date min Weekday
0           0    0         2786.0 2021-06-17   0       3
2647        1    0         1874.0 2021-06-17   0       3
5294        2    0          709.0 2021-06-17   0       3
7941        3    0          645.0 2021-06-17   0       3
10588       4    0          138.0 2021-06-17   0       3
...       ...  ...            ...        ...  ..     ...
5063711  1913    0          339.0 2021-06-17   0       3
5066358  1914    0          221.0 2021-06-17   0       3
5069005  1915    0          327.0 2021-06-17   0       3
5071652  1916    0          327.0 2021-06-17   0       3
5074299  1917    0          338.0 2021-06-17   0       3

[1918 rows x 6 columns]


In [17]:
context_length = 12*60
prediction_length = 2*60

test = TimeSeriesDataSet(
    carpark_data_c,
    time_idx="tid",
    group_ids=["sid"],
    target="AvailableLots",
    time_varying_unknown_reals=["AvailableLots"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["min", "Weekday"],
    predict_mode=True
)

In [18]:
real_prediction=best_model.predict(test, batch_size=500, show_progress_bar=True)

Predict:   0%|          | 0/4 [00:00<?, ? batches/s]

In [19]:
print(real_prediction.shape)

torch.Size([1918, 120])


In [20]:
# Predict the carpark lots using form [ID, 0 min : 30 mins]
# Here predicting carpark ID = 548 for future 30 minutes
real_prediction[548,0:30]

tensor([848.0387, 847.0284, 846.6663, 845.5227, 843.6810, 842.7443, 842.2877,
        842.2467, 840.5222, 839.0427, 838.0204, 837.4323, 837.1828, 836.8389,
        836.0161, 835.5616, 835.3127, 835.1778, 832.2161, 831.1003, 830.5830,
        830.5827, 831.0407, 830.1378, 826.9680, 825.1459, 824.2448, 824.2203,
        821.4538, 821.4397])